In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# ! pip install google-cloud-storage

In [3]:
from io import BytesIO
import pandas as pd
from google.cloud import storage

In [4]:
from google.colab import auth
auth.authenticate_user()

# データ読み込み

In [5]:
client = storage.Client('EasyJapanese')
bucket = client.get_bucket('easyjapanese')
blob = bucket.blob('data/data_pool.csv')
content = blob.download_as_bytes()
df = pd.read_csv(BytesIO(content))
df

,Date,Easy URL,Easy article,Regular URL,Regular article
0,2022-12-22,https://www3.nhk.or.jp/news/easy/k100139301910...,気象庁によると、22日から26日ごろまで日本海側などで雪がたくさん降る心配があります。21日...,https://www3.nhk.or.jp/news/html/20221221/k100...,22日以降、大雪が予想されるとして、国土交通省と気象庁は「大雪に対する緊急発表」を行いました...
1,2022-12-22,https://www3.nhk.or.jp/news/easy/k100139299610...,北陸新幹線は2024年春に福井県まで走るようになる予定です。新幹線がとまる福井駅の前では、観...,https://www3.nhk.or.jp/news/html/20221221/k100...,福井県の魅力をPRしようと、1年余りあとに開業する北陸新幹線の福井駅の近くに、9体の恐竜のモ...
2,2022-12-22,https://www3.nhk.or.jp/news/easy/k100139295110...,将来、南海トラフ地震という地震が起こると、東海地方から九州までの間でとても大きな被害が出ると...,https://www3.nhk.or.jp/news/html/20221221/k100...,南海トラフ地震が起きたとき、被災地で医師が大幅に不足するという新たな調査結果です。東海から九...
3,2022-12-21,https://www3.nhk.or.jp/news/easy/k100139285910...,気象庁は20日、来年1月から3月までの気温や雪などの天気予報を発表しました。1月はペルーの近...,https://www3.nhk.or.jp/news/html/20221220/k100...,気象庁は20日、来年1月から3月にかけての気温や雪などの長期予報を発表しました。日本海側では...
4,2022-12-21,https://www3.nhk.or.jp/news/easy/k100139286710...,ウクライナの首都キーウにある広場で、19日から大きなクリスマスツリーが国の旗と同じ青と黄色に...,https://www3.nhk.or.jp/news/html/20221220/k100...,ウクライナの首都キーウで巨大なクリスマスツリーの点灯が始まり、ロシアによる軍事侵攻が続く中、...
...,...,...,...,...,...
1034,2021-11-26,https://www3.nhk.or.jp/news/easy/k100133620810...,今月8日から留学生などの外国人が日本に入ることができるようになりました。しかし、1日に入る人...,https://www3.nhk.or.jp/news/html/20211125/k100...,新型コロナウイルスの水際対策が緩和された一方で、いまだに、多くの留学生が入国できない状況が続...
1035,2021-11-26,https://www3.nhk.or.jp/news/easy/k100133596110...,毎年冬に、札幌市では凍った道の滑りやすさを知らせる「つるつる予報」が出ます。この予報は、天気...,https://www3.nhk.or.jp/news/html/20211124/k100...,Not Found
1036,2021-11-26,https://www3.nhk.or.jp/news/easy/k100133604110...,お年寄りや障害がある人が生活している施設では、新型コロナウイルスがうつらないように、家族など...,https://www3.nhk.or.jp/news/html/20211125/k100...,厚生労働省は、全国の高齢者施設などに対し、面会を希望する家族がワクチン接種を済ませている場合...
1037,2021-11-25,https://www3.nhk.or.jp/news/easy/k100133597710...,1986年11月21日、東京の伊豆大島で大きな噴火がありました。溶岩が家の近くまで来て、住ん...,https://www3.nhk.or.jp/news/html/20211124/k100...,Not Found


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             1039 non-null   object
 1   Easy URL         1039 non-null   object
 2   Easy article     1039 non-null   object
 3   Regular URL      1039 non-null   object
 4   Regular article  1039 non-null   object
dtypes: object(5)
memory usage: 40.7+ KB


# 正規化処理

In [7]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata
 
def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))
 
    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c
 
    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s
 
def remove_extra_spaces(s):
    s = re.sub('[ ]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'
 
    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s
 
    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s
 
def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)
 
    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}
 
    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))
 
    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

In [8]:
import re
import numpy as np
import pickle
from tqdm import tqdm
 
tag_regex = re.compile(r"<[^>]*?>")
 
def normalize_text(text):
    text = text.replace("\t", " ")
    text = normalize_neologd(text)
    text = tag_regex.sub("", text)
    text = text.replace("&quot;", "\"").replace("&amp;", "&").replace("&lt;", "<").replace("&gt;", ">").replace("&nbsp;", " ")
    text = text.replace(' ', '')
    return text
 
all_data = []
count = 0
keys = ['Easy article', 'Regular article']
for index, data in df.iterrows():
    for key in keys:
      if (
        data[key] == None or data[key] == np.nan 
        or not data[key] or data[key] == 'Unexpected' 
        or data[key] != 'Not Found'
        ):
        continue
    if (
        data[key] != None and data[key] != np.nan 
        and data[key] != 'Unexpected'
        and data[key] != 'Not Found'
        ):
        all_data.append({'text': normalize_text(data[keys[1]]),
                         'response': normalize_text(data[keys[0]]),})

# データセット分割

In [9]:
import random
from tqdm import tqdm
 
FILE_PATH = '/content/drive/MyDrive/Learning/EasyJapanese_analysis/corpus/'
random.seed(1234)
random.shuffle(all_data)
 
def to_line(data):
    text = data['text']
    response = data['response']
 
    assert len(text) > 0 and len(response) > 0
    return f'{text}\t{response}\n'
 
data_size = len(all_data)
train_ratio, val_ratio, test_ratio = 0.95, 0.03, 0.02
 
with open(FILE_PATH+'train.tsv', 'w', encoding='utf-8') as f_train, \
    open(FILE_PATH+'val.tsv', 'w', encoding='utf-8') as f_val, \
    open(FILE_PATH+'test.tsv', 'w', encoding='utf-8') as f_test:
    
    for i, data in tqdm(enumerate(all_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        elif i < (train_ratio + val_ratio) * data_size:
            f_val.write(line)
        else:
            f_test.write(line)

1031it [00:00, 8029.23it/s]


In [10]:
train = pd.read_table(FILE_PATH+'train.tsv', header=None)
train

,0,1
0,訪問介護の現場で働くホームヘルパーなどのうち、4分の1を65歳以上の人が占めていることが厚生...,介護労働安定センターは毎年、介護の仕事をする人について調べています。2021年度は、8500...
1,大手電力10社が国の認可を受けた電気料金は、燃料価格の上昇分が料金に反映できる上限に達してい...,電気の料金がとても高くなっています。普通の家庭が今年12月に使って来年1月に払う料金も、高い...
2,アメリカのアクション映画「ダイ・ハード」などで知られる俳優のブルース・ウィリスさんが、失語症...,アメリカの映画で有名なブルース・ウィリスさんが俳優をやめることがわかりました。家族によると、...
3,JR西日本の運転士が、ミスで仕事が遅れた1分間ぶんの賃金56円を会社から支払われなかったのは...,2020年6月、JR西日本の運転士が電車を車庫に入れるときに間違えて、仕事が終わる時間が1分...
4,多くの人が初詣に訪れる京都市の北野天満宮で、正月に臨時でみこを務める学生らの研修会が行われま...,京都市にある北野天満宮は、勉強の神様の菅原道真で有名な神社です。正月には、とてもたくさんの人...
...,...,...
975,新型コロナウイルスワクチンの3回目の接種率が初めて全国で年代別に公表され、70代以上は80%...,11日の政府の発表によると、新型コロナウイルスの3回目のワクチンを受けた人は日本の人口の45...
976,太平洋戦争の発端となった真珠湾攻撃から8日で80年となります。真珠湾があるハワイのオアフ島で...,1941年12月8日、日本の軍がハワイの真珠湾でアメリカの軍の船などを攻撃して太平洋戦争が始...
977,野菜や果物を多くとる人は、少ない人に比べて亡くなるリスクが10%近く低くなるとする、大規模調...,横浜市立大学などは、40歳から69歳までの約9万5000人について、野菜や果物の効果を調べま...
978,5日の記録的な大雨で道路が寸断された福井県南越前町では9つの集落で車が通れない「孤立状態」が...,今月3日から5日まで、東北地方や北陸地方で雨がとてもたくさん降りました。多くの川で水があふれ...


In [11]:
! head -2 "/content/drive/MyDrive/Learning/EasyJapanese_analysis/corpus/test.tsv"

九州北部と四国、中国地方、近畿、それに北陸ではこのまま確定すれば過去最も早い梅雨明けが発表されました。東京都心では4日連続の猛暑日となるなど、猛暑日は全国の100地点に達しています。29日はさらに気温が上がるところもあり、熱中症に厳重に警戒してください。28日は関東甲信から沖縄にかけて広く高気圧に覆われ、晴れて気温が上がっています。気象庁は28日午前11時、「九州北部と四国、中国地方、近畿、それに北陸が梅雨明けしたとみられる」と発表しました。気象庁によりますと、いずれもこのまま確定すれば統計を取り始めた昭和26年以降、過去最も早い梅雨明けとなります。▽北陸の梅雨明けは平年よりも25日▽九州北部、中国地方、近畿の梅雨明けは平年よりも21日▽四国の梅雨明けは平年よりも19日、いずれも早くなっています。梅雨の期間は▽中国地方、近畿、北陸は14日間▽四国は15日間▽九州北部は17日間で、北陸を除いて過去最も短くなります。これで東北と、梅雨のない北海道を除く地域で梅雨明けしたことになります。28日も各地で気温が上がり、日中の最高気温は▽山梨県甲州市勝沼で38.7度▽岐阜県多治見市で37.9度▽愛知県豊田市で37.8度▽埼玉県鳩山町で37.3度▽富山市で37.1度などと、各地で猛暑日になっています。このほか、東京の都心でも午後1時すぎに35.1度を観測し、6月としては初めて、4日連続で猛暑日となりました。28日、猛暑日となった地点数は100か所に達し、今シーズン最も多くなっています。梅雨明け直後は多くの人が暑さに慣れていないため、熱中症のリスクが高くなります。引き続き熱中症に警戒し▽外出はなるべく避ける▽こまめに水分を補給する▽我慢せず冷房を適切に使用する▽屋外で会話が少ない場面などではマスクを外すなど、対策をとってください。29日はさらに気温が上がるところが多く、関東では40度に達するところもある見込みで、厳重な警戒が必要です。ことしもすでに、高齢者などが熱中症の疑いで死亡するケースが確認されています。屋内では適切に冷房を使用するよう、離れて住む家族の方や周囲の人が声をかけるようにしてください。	気象庁は28日、九州北部と四国、中国地方、近畿、北陸で梅雨が終わったようだと言いました。このあと天気が変わることがなかったら、今まででいちばん早く梅雨が終わった年になります。関東など

# 学習に必要なクラス等の定義

In [12]:
! pip install -qU transformers[ja] pytorch_lightning sentencepiece torch==1.10.0+cu111 torchvision==0.11.1+cu111 torchaudio torchtext -f https://download.pytorch.org/whl/torch_stable.html

     |████████████▌                   | 834.1 MB 1.4 MB/s eta 0:15:33tcmalloc: large alloc 1147494400 bytes == 0x3a3da000 @  0x7f9d09ca7615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 0x49abe4
     |███████████████▉                | 1055.7 MB 61.7 MB/s eta 0:00:18tcmalloc: large alloc 1434370048 bytes == 0x7ea30000 @  0x7f9d09ca7615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 0x49abe4
     |████████████████████            | 1336.2 MB 1.2 MB/s eta 0:10:54tcmalloc: large alloc 1792966656 bytes == 0x3862000 @  0x7f9d09ca7615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x

In [13]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

import sentencepiece

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
    )

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(1013)
     

In [14]:

# 事前学習済みモデル
PRETRAINED_MODEL_NAME = 'megagonlabs/t5-base-japanese-web'

# 転移学習済みモデルを保存する場所
MODEL_DIR = '/content/drive/MyDrive/Learning/EasyJapanese_analysis/model/'

# その他ハイパーパラメータ

In [15]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir=FILE_PATH,  # データセットのディレクトリ
    model_name_or_path=PRETRAINED_MODEL_NAME,
    tokenizer_name_or_path=PRETRAINED_MODEL_NAME,

    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    gradient_accumulation_steps=1,

    # max_input_length=512,
    # max_target_length=4,
    # train_batch_size=8,
    # eval_batch_size=8,
    # num_train_epochs=4,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    # amp_backend='apex', # 追加
    max_grad_norm=1.0,
    seed=42,
)


# データセット変換

In [16]:
class TsvDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=1024, target_max_len=256):
        self.file_path = os.path.join(data_dir, type_path)
        
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()
  
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        source_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": source_mask, 
                "target_ids": target_ids, "target_mask": target_mask}

    def _make_record(self, regular, easy):
        # ニュースタイトル生成タスク用の入出力形式に変換する。
        input = f"{regular}"
        target = f"{easy}"
        return input, target
  
    def _build(self):
        with open(self.file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip().split("\t")
                assert len(line) == 2
                assert len(line[0]) > 0
                assert len(line[1]) > 0

                regular = line[0]
                easy = line[1]

                input, target = self._make_record(regular, easy)

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    [input], max_length=self.input_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = self.tokenizer.batch_encode_plus(
                    [target], max_length=self.target_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                self.inputs.append(tokenized_inputs)
                self.targets.append(tokenized_targets)


In [17]:
# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME, is_fast=True)

# テストデータセットの読み込み
train_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "train.tsv", 
                           input_max_len=700, target_max_len=200)

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

訓練データの１レコード目を試しに見てみる。

In [18]:
for data in train_dataset:
    print("A. 入力データの元になる文字列")
    print(tokenizer.decode(data["source_ids"]))
    print()
    print("B. 入力データ（Aの文字列がトークナイズされたトークンID列）")
    print(data["source_ids"])
    print()
    print("C. 出力データの元になる文字列")
    print(tokenizer.decode(data["target_ids"]))
    print()
    print("D. 出力データ（Cの文字列がトークナイズされたトークンID列）")
    print(data["target_ids"])
    break

A. 入力データの元になる文字列
訪問介護の現場で働くホームヘルパーなどのうち、4分の1を65歳以上の人が占めていることが厚生労働省所管の財団法人の調査で分かりました。この調査は厚生労働省が所管する財団法人「介護労働安定センター」が毎年行っているもので、昨年度(令和3年度)は全国のおよそ8500の事業所から回答を得ました。それによりますと、事業所で働く人合わせて15万5000人余りのうち65歳以上の人は2万1342人で、全体に占める割合は13.7%となりました。前回の調査(令和2年度)から1.4ポイントの増加となりました。職種別で65歳以上の人の割合を見ると、理学療法士や作業療法士などが1.7%だったのに対し、ホームヘルパーなど訪問介護員が最も高い25.4%に上りました。一方、介護職で働く人のうち、新型コロナウイルスなどの感染症やけがなど健康面の不安があると答えた人は昨年度から7.6ポイント増え、28.1%に上りました。調査では、介護職の中でも特に人手不足が深刻だと指摘されている訪問介護の現場が、健康面に不安を抱えた高齢のヘルパーに支えられている実態が浮き彫りとなりました。東京北区で11年間ホームヘルパーとして働いている荒木美佐子さんは、長く働くことができる仕事だと50代で資格を取得しました。ことし69歳になりましたが、今も一日5件から7件の訪問介護を週に6日ほど行っています。健康への不安や体力の衰えはあるものの、訪問介護の仕事にやりがいも感じています。荒木さんは、「利用者も自分も高齢なので新型コロナウイルスに感染してしまうと大変ですが、元気であれば定年がない仕事なので、細く長く働ければと思います」と話しています。荒木さんが所属している訪問介護事業所では、15人のホームヘルパーのうち8人が65歳以上です。事業所によりますと、荒木さんのように経験が豊富な高齢のヘルパーは即戦力になる一方で、求人を出しても若い世代からの応募はほとんどないのが実情だと言います。さらに最近は新型コロナのクラスターで通っていた施設が利用できなくなった人からの訪問介護の依頼が増えていると言い、事業所では今後の事業継続に危機感を募らせています。職員の黒澤加代子さんは、「65歳以上のヘルパーの中には、腰痛があったり、持病の薬を飲みながら働いてくれている人もいて、数年後どころか、来週大丈夫だろうかと、

# 学習処理クラス

In [19]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hps):
        super().__init__()
        self.hps = hps

        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(hps.model_name_or_path)

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(hps.tokenizer_name_or_path, is_fast=True)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]

        # All labels set to -100 are ignored (masked), 
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        loss = self._step(batch)
        self.log("train_loss", loss)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        loss = self._step(batch)
        self.log("val_loss", loss)
        return {"val_loss": loss}

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        loss = self._step(batch)
        self.log("test_loss", loss)
        return {"test_loss": loss}

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hps.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.hps.learning_rate, 
                          eps=self.hps.adam_epsilon)
        self.optimizer = optimizer

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.hps.warmup_steps, 
            num_training_steps=self.t_total
        )
        self.scheduler = scheduler

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return TsvDataset(
            tokenizer=tokenizer, 
            data_dir=args.data_dir, 
            type_path=type_path, 
            input_max_len=args.max_input_length,
            target_max_len=args.max_target_length)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                             type_path="train.tsv", args=self.hps)
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="val.tsv", args=self.hps)
            self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.hps.train_batch_size * max(1, self.hps.n_gpu)))
                // self.hps.gradient_accumulation_steps
                * float(self.hps.num_train_epochs)
            )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, 
                          batch_size=self.hps.train_batch_size, 
                          drop_last=True, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, 
                          batch_size=self.hps.eval_batch_size, 
                          num_workers=4)


# 転移学習を実行

In [20]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  700,  # 入力文の最大トークン数
    "max_target_length": 200,  # 出力文の最大トークン数
    "train_batch_size":  2,
    "eval_batch_size":   2,
    "num_train_epochs":  4,
    })
args = argparse.Namespace(**args_dict)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    amp_backend='apex',
    gradient_clip_val=args.max_grad_norm,
    default_root_dir=f"{MODEL_DIR}/checkpoint",
)

In [21]:
# 転移学習の実行（GPUを利用すれば1エポック10分程度）
model = T5FineTuner(args)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

# 最終エポックのモデルを保存
model.tokenizer.save_pretrained(MODEL_DIR)
model.model.save_pretrained(MODEL_DIR)

del model

Downloading:   0%|          | 0.00/631 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/990M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:182: LightningDeprecationWarning: Setting `amp_level` inside the `Trainer` is deprecated in v1.8.0 and will be removed in v1.10.0. Please set it inside the specific precision plugin and pass it to the `Trainer`.
  rank_zero_deprecation(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=4` reached.


# 学習済みモデルの読み込み

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

# 全テストデータに対する本文の生成

In [22]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

# テストデータの読み込み
test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "test.tsv", 
                          input_max_len=args.max_input_length, 
                          target_max_len=args.max_target_length)

test_loader = DataLoader(test_dataset, batch_size=8, num_workers=4)

trained_model.eval()

inputs = []
outputs = []
targets = []

for batch in tqdm(test_loader):
    input_ids = batch['source_ids']
    input_mask = batch['source_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    output = trained_model.generate(input_ids=input_ids, 
        attention_mask=input_mask, 
        max_length=args.max_target_length,
        repetition_penalty=10.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        )

    output_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                            clean_up_tokenization_spaces=False) 
                for ids in output]
    target_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                               clean_up_tokenization_spaces=False) 
                for ids in batch["target_ids"]]
    input_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                               clean_up_tokenization_spaces=False) 
                for ids in input_ids]

    inputs.extend(input_text)
    outputs.extend(output_text)
    targets.extend(target_text)
    

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
for output, target, input in zip(outputs, targets, inputs):
    print("Regular:     " + input)
    print("generated: " + output)
    print("actual:    " + target)
    print()

Regular:     九州北部と四国、中国地方、近畿、それに北陸ではこのまま確定すれば過去最も早い梅雨明けが発表されました。東京都心では4日連続の猛暑日となるなど、猛暑日は全国の100地点に達しています。29日はさらに気温が上がるところもあり、熱中症に厳重に警戒してください。28日は関東甲信から沖縄にかけて広く高気圧に覆われ、晴れて気温が上がっています。気象庁は28日午前11時、「九州北部と四国、中国地方、近畿、それに北陸が梅雨明けしたとみられる」と発表しました。気象庁によりますと、いずれもこのまま確定すれば統計を取り始めた昭和26年以降、過去最も早い梅雨明けとなります。▽北陸の梅雨明けは平年よりも25日▽九州北部、中国地方、近畿の梅雨明けは平年よりも21日▽四国の梅雨明けは平年よりも19日、いずれも早くなっています。梅雨の期間は▽中国地方、近畿、北陸は14日間▽四国は15日間▽九州北部は17日間で、北陸を除いて過去最も短くなります。これで東北と、梅雨のない北海道を除く地域で梅雨明けしたことになります。28日も各地で気温が上がり、日中の最高気温は▽山梨県甲州市勝沼で38.7度▽岐阜県多治見市で37.9度▽愛知県豊田市で37.8度▽埼玉県鳩山町で37.3度▽富山市で37.1度などと、各地で猛暑日になっています。このほか、東京の都心でも午後1時すぎに35.1度を観測し、6月としては初めて、4日連続で猛暑日となりました。28日、猛暑日となった地点数は100か所に達し、今シーズン最も多くなっています。梅雨明け直後は多くの人が暑さに慣れていないため、熱中症のリスクが高くなります。引き続き熱中症に警戒し▽外出はなるべく避ける▽こまめに水分を補給する▽我慢せず冷房を適切に使用する▽屋外で会話が少ない場面などではマスクを外すなど、対策をとってください。29日はさらに気温が上がるところが多く、関東では40度に達するところもある見込みで、厳重な警戒が必要です。ことしもすでに、高齢者などが熱中症の疑いで死亡するケースが確認されています。屋内では適切に冷房を使用するよう、離れて住む家族の方や周囲の人が声をかけるようにしてください。
generated: 北海道、青森県から沖縄県では晴れて気温が上がっています。気象庁は28日、「九州北部と四国地方で梅雨が始まりました」と言います。『北陸地

In [33]:
import pandas as pd
RESULT_PATH = '/content/drive/MyDrive/Learning/EasyJapanese_analysis/result/'

pd.Series(inputs).to_csv(os.path.join(RESULT_PATH, 'inputs.csv'), index=False)
pd.Series(outputs).to_csv(os.path.join(RESULT_PATH, 'outputs.csv'), index=False)
pd.Series(targets).to_csv(os.path.join(RESULT_PATH, 'targets.csv'), index=False)

In [61]:
ops = pd.read_csv(os.path.join(RESULT_PATH, 'outputs.csv')).values.flatten()
print(ops)

['北海道、青森県から沖縄県では晴れて気温が上がっています。気象庁は28日、「九州北部と四国地方で梅雨が始まりました」と言います。『北陸地方の水稲の畑』という本を読んだ人は「暑さに慣れていないため熱中症になる危険が高くなりそうです)そして外に出ないようにしてくださいね」「エアコンを使うなど気をつけることが必要になりません」、「マスクをしておいても大丈夫だと思います)」と言っていました。2017年6月になってからいちばん早く始まりそうな記録になっていて、2019年から今までで最も長くなっていています。2018年の7月から8月に続けて3回連続で猛夏が続きたいと考えていてとても心配してくれているようでもあります 29日の午前11時ごろまでには100以上の所がありまして大変だと専門家などが言っておきましょう'
 'ロシアがアフガニスタンに攻撃を続けています。宮崎駿監督の「となりのトトロ」は、自然の中で生活する姉妹と不思議な生き物たちと交流しながら話します。1987年に映画が出ました。1988年には大きな人気になります。1989年にもたくさんの映像をアメリカで出し続けていました。『としのトトロ』では、「いきもの」というキャラクターについて考えていきながら成長していく物語です。1996年に初めて日本のアニメ映画の『となりのトトロ』(1988年の映画)もイギリスで上演することになりました。2017年のアニメーションで有名な久石譲さんが初めて舞台に立つことになりそうだと言っています。2011年から今年の間イギリスの劇場で公演がありそうなことがわかりましてとても楽しみにしていませんか?'
 'ロシアがウクライナを攻撃しているため、国を追われる人が増えています。20日は神奈川県鎌倉市の大船観音寺で「世界難民の日」にしました。2019年4月1日に国連がつくった世界の平和のためのイベントがありまして、「世界で困っている人たちのために祈ります!」と話し合っていました。『世界に生きるために祈る会』という団体も一緒に行いませんかと言っています。2011年に亡くなった人の像などと一緒にこの行事を行いたいと考えていて、2019年のクリスマスにはたくさんのお祝いをする予定です。2017年には日本に住む外国人たちがユネスコの世界大使になることが決まりそうだと言い始めていました。2018年からはロシアの大